Изменение docker-compose

In [2]:
docker_compose_content = """
version: '3.7'
services:
    minio:
        restart: always
        image: minio/minio@sha256:6d6cf693fd70ca6e15709fa44d39b44f98fc5b58795697486a95ac1cc2ad9880
        container_name: mlflow_s3
        ports:
            - "9000:9000"
            - "9001:9001"
        command: server /data --console-address ':9001' --address ':9000'
        environment:
            - MINIO_ROOT_USER=${AWS_ACCESS_KEY_ID}
            - MINIO_ROOT_PASSWORD=${AWS_SECRET_ACCESS_KEY}
        volumes:
            - minio_data:/data
        networks:
            - mlflow_net

    mc:
        image: minio/mc@sha256:3455a7bae6058ea83f797a95c0e29a4daedff6f79b1f87a0ede429e0344734ab
        depends_on:
            - minio
        container_name: mc
        env_file:
            - .env
        entrypoint: >
            /bin/sh -c "
            /tmp/wait-for-it.sh minio:9000 &&
            /usr/bin/mc alias set minio http://minio:9000 ${AWS_ACCESS_KEY_ID} ${AWS_SECRET_ACCESS_KEY} &&
            /usr/bin/mc mb minio/mlflow;
            exit 0;
            "
        volumes:
            - ./wait-for-it.sh:/tmp/wait-for-it.sh

    db:
        restart: always
        image: mysql/mysql-server@sha256:5b40d96b11333570143d98d3a74100fefadb9abb17b27a95dbc9ad33544ec142
        container_name: mlflow_db
        ports:
            - "54321:3308"
        environment:
            - MYSQL_DATABASE=${MYSQL_DATABASE}
            - MYSQL_USER=${MYSQL_USER}
            - MYSQL_PASSWORD=${MYSQL_PASSWORD}
            - MYSQL_ROOT_PASSWORD=${MYSQL_ROOT_PASSWORD}
        volumes:
            - dbdata:/var/lib/mysql
        networks:
            - mlflow_net

    web:
        restart: always
        build: ./mlflow
        image: mlflow_server
        container_name: mlflow_server
        depends_on:
            - mc
            - db
        ports:
            - "5000:5000"
        environment:
            - MLFLOW_S3_ENDPOINT_URL=http://minio:9000
            - AWS_ACCESS_KEY_ID=${AWS_ACCESS_KEY_ID}
            - AWS_SECRET_ACCESS_KEY=${AWS_SECRET_ACCESS_KEY}
        command: mlflow server --backend-store-uri mysql+pymysql://${MYSQL_USER}:${MYSQL_PASSWORD}@db:3306/${MYSQL_DATABASE} --default-artifact-root s3://mlflow/ --host 0.0.0.0
        networks:
            - mlflow_net
        
volumes:
    dbdata:
    minio_data:

networks:
  mlflow_net:
    driver: bridge
"""

with open("mlflow-server/docker-compose.yml", "w") as file:
    file.write(docker_compose_content)


Запуск докера

In [3]:
!cd mlflow-server && docker-compose up -d --build

[+] Building 0.0s (0/0)                                          docker:default
[+] Building 0.0s (0/0)                                          docker:default
[+] Building 0.0s (0/0)                                          docker:default
[+] Building 0.1s (1/2)                                          docker:default
 => [web internal] load .dockerignore                                      0.1s
 => => transferring context: 2B                                            0.0s
 => [web internal] load build definition from Dockerfile                   0.1s
 => => transferring dockerfile:                                            0.1s
[+] Building 0.2s (2/3)                                          docker:default
 => [web internal] load .dockerignore                                      0.1s
 => => transferring context: 2B                                            0.0s
 => [web internal] load build definition from Dockerfile                   0.1s
 => => transferring dockerfile: 163B    